# Implementando Michigrad

In [ ]:
import torch
import numpy as np
from graphviz import Digraph


In [ ]:
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def show_graph(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{%s | data %.4f | grad %.4f}" % (n.name, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot
    
class Value():
    def __init__(self, data, _children=(), _op="", name=""):
        self.data = data
        self.name = name
        self.grad = 0.0
        self._backward = lambda: None
        self._op = _op
        self._prev = set(_children)

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self,other), "+")
        
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self,other), "*")

        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "solamente soportamos potencias de int/float"
        out = Value(self.data ** other, _children=(self, ), _op=f"**{other}")

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        return self * other ** -1

    def exp(self):
        out = np.exp(self.data)
        out = Value(out, _children=(self,), _op="exp")
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward       
        return out

    def tanh(self):
        x = self.data
        t = (np.e ** (2*x) - 1)/(np.e ** (2*x) + 1)
        out = Value(t, _children=(self, ), _op="tanh")

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v is not visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.
        for node in reversed(topo):
            node._backward()

    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __rsub__(self, other):
        return other + (-self)

    def __repr__(self):
        return f'Value(data={self.data}, grad={self.grad}, _op="{self._op}", name={self.name})'

In [ ]:
W0 = Value(-.5, name='W₀')
x0 = Value(.5, name='x₀')
W1 = Value(-.3, name='W₁')
x1 = Value(2, name='x₁')
W0x0 = W0 * x0; W0x0.name = 'W0x0'
W1x1 = W1 * x1; W1x1.name = 'W1x1'
yhat = W0x0 + W1x1; yhat.name = 'ŷ'
yhat = yhat.tanh(); yhat.name = 'tanh(ŷ)'
y = Value(.8, name='y')
L = y - yhat; L.name = 'L'

## Conclusiones
* Hemos creado una pequeña librería para representar MLPs.
* La librería es suficientemente pontente para representar un clasificador sencillo.
* Es fácil de extender y la interfáz es en su mayor parte compatible con PyTorch.
* Lo visto debería alcanzar para comprender totalmente el código de Michigrad y Micrograd.

## Ejercicios
* Implementar el parámetro `bias` en `Neuron` y `Layer` para poder crear neuronas sin sesgo. (Fácil)
  * Pista: mirar la implementación de Michigrad.
* Implementar el modelo `xor` usando PyTorch. (Fácil si usaste Pytorch antes, Intermedio si no lo hiciste)
  * Pista: PyTorch no implementa un MLP. Se puede redefirnir la clase `MLP` usando `torch.Linear` en lugar de capas de neuronas (`Layer` y `Neuron`).
* Implementar las funciones de activación como capas. (Intermedio)
  * Pista: Reemplazar `Layer` por `Linear`, y definir una clase por cada función de activación. La capa `Linear` se comportará como una capa de neuronas y la capa de activación aplicará la función de activación a cada salida de todas las neuronas de la capa anterior.
* Implementar la clase Module que permita crear modelos como listas de modulos. Todos los módulos deben soportar la `__call__(self, x)` que hace la forward pass.  (Difícil)
  * Pista: Se tendría que poder crear un modelo como una lista de modulos, así:
   ```python
       model = Model([Linear(2, 4), Linear(4, 4, bias=False), Linear(4, 3), Tanh(3)])
       model(x)
   ```